In [1]:
# load the dataset into an RDD to get started
input_rdd = sc.textFile("/FileStore/tables/movielens/movies.csv")

In [2]:
# we will extract the year with this function. if there is problem with our data we will just return a None value
import re
def get_year(name):
    year = None
    try:
      pattern = re.compile(r"\((\d+)\)")
      year = int(pattern.findall(name)[0])
    except ValueError:
      pass
    except IndexError:
      pass
    
    return year

In [3]:
movie_info_rdd = input_rdd.filter(lambda line: 'movieId' not in line)
movie_info_list_rdd = movie_info_rdd.map(lambda x: x.split(','))

In [4]:
# we can use the map operation to apply our custom function to every name in our rdd
movie_year_rdd = movie_info_list_rdd.map(lambda x: get_year(x[1]))

### Actions
Actions are RDD methods that return a value to a driver program. This section discusses the commonly used
RDD actions. Spark will only "go to work" when you apply an action on top of your RDD.

##### collect
The collect method returns the elements in the source RDD as an array. This method should be used with
caution since it moves data from all the worker nodes to the driver program. It can crash the driver program
if called on a very large RDD.

In [7]:
# lets print out all the different years we have movies from in our data
movie_year_rdd.distinct().collect()

##### count
The count method returns a count of the elements in the source RDD

In [9]:
# count the number of movies we have in our data
movie_year_rdd.count()

##### countByValue
The countByValue method returns a count of each unique element in the source RDD. It returns an instance
of the Map class containing each unique element and its count as a key-value pair.

In [11]:
# count the number of movies we have from every year
movie_year_rdd.countByValue()

##### first
The first method returns the first element in the source RDD

In [13]:
movie_year_rdd.first()

##### max
The max method returns the largest element in an RDD

In [15]:
movie_year_rdd.max()

##### min
The min method returns the smallest element in an RDD.

In [17]:
movie_year_rdd.filter(lambda x: x is not None).min()

##### take
The take method takes an integer N as input and returns an array containing the first N element in the
source RDD.

In [19]:
movie_year_rdd.take(10)

##### takeOrdered
The takeOrdered method takes an integer N as input and returns an array containing the N smallest
elements in the source RDD

In [21]:
movie_year_rdd.filter(lambda x: x is not None).takeOrdered(10)

In [22]:
movie_year_rdd.filter(lambda x: x is not None).distinct().takeOrdered(5, key = lambda x: -x)

##### top
The top method takes an integer N as input and returns an array containing the N largest elements in the
source RDD.

In [24]:
movie_year_rdd.filter(lambda x: x is not None).top(3)

### Actions on RDD of Numeric Types

RDDs containing data elements of type Integer, Long, Float, or Double support a few additional actions that are useful for statistical analysis. The commonly used actions from this group are briefly described next

##### mean
The mean method returns the average of the elements in the source RDD.

In [27]:
sc.parallelize([1,2,3,4,5,6,7,8,9,10]).mean()

##### stdev
The stdev method returns the standard deviation of the elements in the source RDD.

In [29]:
sc.parallelize([1,2,3,4,5,6,7,8,9,10]).stdev()

##### sum
The sum method returns the sum of the elements in the source RDD

In [31]:
sc.parallelize([1,2,3,4,5,6,7,8,9,10]).sum()

#### variance
The variance method returns the variance of the elements in the source RDD

In [33]:
sc.parallelize([1,2,3,4,5,6,7,8,9,10]).sum()

### Saving an RDD

##### saveAsTextFile
The saveAsTextFile method saves the elements of the source RDD in the specified directory on any
Hadoop-supported file system. Each RDD element is converted to its string representation and stored as a
line of text

##### saveAsObjectFile
The saveAsObjectFile method saves the elements of the source RDD as serialized Java objects in the specified directory.

##### saveAsSequenceFile
The saveAsSequenceFile method saves an RDD of key-value pairs in SequenceFile format. An RDD of keyvalue pairs can also be saved in text format using the saveAsTextFile.